In [2]:
import torch

torch.cuda.is_available()

True

In [2]:
import polars as pl
from minio import Minio
from datasets import load_dataset
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer

In [5]:
model_name = "google/madlad400-3b-mt"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda")

model

T5ForConditionalGeneration(
  (shared): Embedding(256000, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(256000, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=2048, bias=False)
              (k): Linear(in_features=1024, out_features=2048, bias=False)
              (v): Linear(in_features=1024, out_features=2048, bias=False)
              (o): Linear(in_features=2048, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1024, out_features=8192, bias=False)
              (wi_1): Linear(in_features=1024, out_features=8192, bias=False)
     

In [1]:
import polars as pl
from minio import Minio
from datasets import load_dataset
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer

In [2]:
from transformers import pipeline

task = "translation"

model_name = "google/madlad400-3b-mt"

translator = pipeline(task, model=model_name, max_length=1024, device="cuda")

translator

In [3]:
translator("<2en> A compra foi realizada facilmente. A entrega foi efetuada muito antes do prazo dado. O produto já começou a ser usado e até o presente, sem problemas.")

[{'translation_text': 'The purchase was made easily. The delivery was made well before the deadline given.The product has already begun to be used and to the present, without problems.'}]

In [5]:
client = Minio(endpoint="storage.io", access_key="admin", secret_key="password", secure=False)

response = client.get_object("ecommerce", "raw/order_reviews.csv")

df = pl.read_csv(response.read())

df.head(200).write_csv("/tmp/order_reviews.head.csv")

dataset = load_dataset("csv", data_files="/tmp/order_reviews.head.csv", streaming=True)

dataset

IterableDatasetDict({
    train: IterableDataset({
        features: Unknown,
        n_shards: 1
    })
})

In [6]:
def translate(row):
    title = row["review_comment_title"]
    message = row["review_comment_message"]

    prompt = "<2en> {text}"
    translated_title = title and translator(prompt.format(text=title))[0]["translation_text"]
    translated_message = message and translator(prompt.format(text=message))[0]["translation_text"]

    row["translated_review_comment_title"] = translated_title
    row["translated_review_comment_message"] = translated_message

    return row

In [7]:
translated_dataset = dataset["train"].map(translate)

translated_dataset

IterableDataset({
    features: Unknown,
    n_shards: 1
})